![giraffe](https://raw.githubusercontent.com/jeinson/jeinson.github.io/master/images/giraffe.jpg)

# Lab 5: Feed Forward Neural Networks

In this lab activity, we will experiment with a simple yet powerful tool called a Feedforward Neural Network, implemented in (you guessed it) pytorch. This network is more complicated than linear and logistic regression that we've seen earlier, but it is a simpler form of neural network because information is only passed forward, i.e. nodes to not form a cycle. (spoiler alert, this will come later) A canonical FFNN is composed of layers of linear nodes called *perceptrons*, which take the value 1 or -1, according to some activation function, dependent on the previous layer. The parameters of a FFNN are learned exactly the same way we learned the parameters for linear and logistic regression, by gradient descent using **back propagation**

![nn_image](https://upload.wikimedia.org/wikipedia/en/5/54/Feed_forward_neural_net.gif)

As it's a right of passage for beginner machine learners, we will start with the MNIST dataset, which consists of scanned images of handwritten digits. The neural net will classify each image as being a 0 through 9. (And hopefully better job than Patrick at classifying bubble buddies)

### Step 1: Load the Training Data

Fortunately MNIST comes in the box with the pytorch package, and has been formatted for input into a neural network. Wrangling your data is half the battle, and it's already done for you :-)

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [ ]:
train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

### Step 2: Making the dataset more manageable

**Batch sizes and iterations**

Using the entire thing to train the model is unadvisable because it 1) would require a ton of RAM and 2.) could result in unstable training, as errors would accumlate very quickly. Therefore, we divide the training data into smaller batches, and feed it through the network iteratively. 

How big is the training dataset? If we want a batch size of 100 images, how many iterations do we need to have so that the network sees each image at least once?

In [ ]:
#### your code here ###

**Epochs**

The formal definition of "epoch" is one complete pass of all the data through the model. How many iterations do we need to get to 5 epochs?

In [ ]:
#### your code here (if you need help multiplying) ###

**Bringing it all together**

Now let's specify the batch size, number of iterations, and the number ofr epochs

In [ ]:
batch_size = 100
n_iters =  # Your code here
n_epochs = #### your code here (but it should be 5 cuz I said so) ####

Define `train_loader` and `test_loader` as one of the objects in the [torch.utils.data](https://pytorch.org/docs/stable/data.html) module. These things will load images from your dataset objects in the batch size you specify. This is one of the flagship features of Pytorch, since loading huge datasets can eat up a ton of memory. Read more about data loaders and how to use them [here](https://github.com/utkuozbulak/pytorch-custom-dataset-examples). Note you should shuffle your training data, but no need to shuffle the test data. 

In [ ]:
#### your code here ####

### Step 3: Create a Model Class

A feed forward neural net looks remarkably similar to the linear and logistic regression networks we played with for the past two weeks. The only difference is an extra linear and non-linear layer. 
![ExtraLayer](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/images/logistic_regression_comparison_nn5.png)

The syntax for defining a neural net in pytorch should be familiar to you by now, but remember to check the API here if you get stuck

In this net, we set the dimensions of each layer as parameters that are specified when defining a model `nn.Module` object. 

In [ ]:
class FeedforwardNeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetwork, self).__init__()
        
        # Linear Function
        #### your code here ####
        
        # Non-linearity (Use a sigmoid for now)
        #### your code here ####
        
        # Another linear function (the readout)
        #### your code here ####
    
    # Here's where we connect all the layers together
    def forward(self, x):
        # Linear function
        #### your code here ####
        
        # Non-linearity 
        #### your code here ####
        
        # The final linear function
        #### your code here ####
        
        # aaaaannnddd spit out the output
        return out

### Step 4: Instantiate the Model Class

Our input size is determined by the size of the image (numbers ranging from 0 to 9) which has a width of 28 pixels and a height of 28 pixels. Hence the size of our input is 784 (28 x 28).

Our output size is what we are trying to predict. When we pass an image to our model, it will try to predict if it's 0, 1, 2, 3, 4, 5, 6, 7, 8, or 9. How many possible digits are there?

Now the tricky part is in determining our hidden layer size, that is the size of our first linear layer prior to the non-linear layer. This can be any number; a larger number implies a bigger model with more parameters. Intuitively we think a bigger model equates to a better model, but a bigger model requires more training samples to learn and converge to a good model (also called curse of dimensionality). Hence, it is wise to pick the model size for the problem at hand. Because it is a simple problem of recognizing digits, we typically would not need a big model to achieve state-of-the-art results.

On the flipside, too small of a hidden size would mean there would be insufficient model capacity to predict competently. In layman terms, too small of a capacity implies a smaller brain capacity so no matter how many training samples you give it, it has a maximum capacity in terms of its predictive power.

In [ ]:
input_dim = # Your code here
hidden_dim =  # Your code here (Try tinkering with this if stuff goes wrong)
output_dim =  # Your code here

model = #### your code here ####

In [ ]:
# If you have a GPU, this is where you can use it!
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

### Step 5: Instantiate a Loss Class

For classification problems, we use cross entropy as a loss function. Don't worry too much about what it actually does. Just define the appropriate object from the `torch.nn` module. 

In [ ]:
#### your code here ####

### Step 6: Instantitate an Optimizer Class

In terms of updating your parameters, we still follow the basic premace of $\theta_{t+1} = \theta_{t} - \eta \cdot \nabla_{\theta_t}$, where $\theta$ are the model parameters, $\eta$ is the learning rate and $\nabla_\theta$ is the gradient. For now we're gonna stick with stochastic gradient descent for the optimizer. It's pretty much the duct tape of ML. As for a learning rate, start with 0.1, but feel free to change it up later if you need. 

In [ ]:
learning_rate = #### your code here ###
optimizer =  #### your code here ###

![T_time](https://media1.tenor.com/images/b04545d91916611c2c2cd61cc3854d2d/tenor.gif)

<center>Time Out!</center>

Take a few minutes to understand what's going on at this point. First off, how many parameters does each layer of the network have? What values to the parameters take when initializing the model? Do you see how at the core, we're really just doing matrix the same operations as in regression? The diagram below shows the interaction amongst our input $X$ and out linear layers' parameters $A_1$, $B_1$, $A_2$, and $B_2$ to reach the final size of 10 x 1.

![net_image](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/images/nn1_params3.png)

### Step 7: Train the Model

This may be the most confusing part, especially accessing data from your training data loader object. To get a feel for how it works, make a plot of the first 25 digits in the first batch of MNIST images, and label the x axis with the true label. Make sure to use the `train_loader` object. 

In [ ]:
import matplotlib.pyplot as plt
#### your code here ###

Training follows the same process as we've seen previously 
1. Convert inputs to tensors with gradient accumulation capabilities
2. Zero the gradients
3. Get a new output given the current parameters
4. Calculate the loss 
5. Get new gradients w.r.t. the model parameters
6. Update the parameters using the gradients, in a direction toward the optimal solution
7. *REPEAT!!*

Now fill in the missing code to train the model. Depending on how powerful your CPU (or GPU) is, training may take a few minutes. For debugging, try reducing the number of epochs or batch size, just to make sure stuff is working properly. 

In [ ]:
iter = 0
for epochs in range(n_epochs):
    
    # This will load images 100 at a time, as specified when you defined this thing
    for i, (images, labels) in enumerate(train_loader):
        
        # Flatten the images to a torch tensor of 1x(28*28) vectors
        # (try the .view method)
        # And make sure they accumulate gradients!
        #### your code here ####
        
        # Clear gradients w.r.t. parameters
        #### your code here ####
        
        # Forward pass to get the output/logits
        #### your code here ####
        
        # Calculate Loss
        #### your code here ####
        
        # Get gradients w.r.t. the parameters
        #### your code here ####
        
        # Update the parameters
        #### your code here ####
        
        iter += 1
        
        # Every 500 iterations, check up on how the model is doing, 
        # by printing the loss and the training accuracy on the held out data. 
        # Accuracy = (number of images correctly identified) / (total number of images)
        #### your code here ####
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images with gradient accumulation capabilities

                # Forward pass only to get logits/output

                # Get predictions from the maximum value

                # Get the total number of labels

                # Calculate the total correct predictions

            accuracy = #### your code here ####

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))
        

Nice work! You trained a neural net! If you got a training accuracy of over 90%, you can reward yourself with a picture of my cat helping me write this lab...

<center><form method="get" action="https://raw.githubusercontent.com/jeinson/jeinson.github.io/master/images/20190220_095041.jpg">
    <button type="submit">Click here for LULz</button>
</form></center>

## Bonus challenge

From the last round of training, select the images that the neural net failed to classify correctly and plot them. Why do you think the net was unable to correctly identify these digits?

In [ ]:
#### your code here ####

*Citation*:
This lab was developed from www.deeplearningwizard.com. With some googling, I'm sure you can find it, but if you copy-paste the answers you won't really learn anything!